In [3]:
import pyodbc
import pandas as pd
import numpy as np
import requests, zipfile, io, os, warnings
from pathlib import Path
from datetime import datetime

warnings.filterwarnings('ignore')

# --- 1. DATABASE CONFIGURATION ---
DB_CONFIG = {
    'server': 'DESKTOP-PG3CATF',
    'database': 'OSHE_Analytics',
    'driver': 'ODBC Driver 18 for SQL Server'
}

# Tambahkan definisi tabel di sini agar mudah diatur (Centralized Mapping)
STAGING_TABLE = 'staging.accidents_raw'
PRODUCTION_TABLE = 'dbo.Accidents_Clustered_Final'
FINAL_TABLE = 'dbo.Accidents_Clustered_Final'

# --- 2. DIRECTORY STRUCTURE ---
BASE_DIR = Path.cwd().parent if Path.cwd().name == 'python' else Path.cwd()
RAW_DIR = BASE_DIR / 'data' / 'raw'
STAGING_DIR = BASE_DIR / 'data' / 'staging'
PROCESSED_DIR = BASE_DIR / 'data' / 'processed'

# Pastikan folder fisik tersedia di Windows Explorer
for folder in [RAW_DIR, STAGING_DIR, PROCESSED_DIR]:
    folder.mkdir(parents=True, exist_ok=True)

# --- 3. CONNECTION HELPER ---
def get_conn_str(db='master'):
    """Fungsi untuk membuat koneksi string secara dinamis."""
    target_db = db if db == 'master' else DB_CONFIG['database']
    return (f"DRIVER={{{DB_CONFIG['driver']}}};"
            f"SERVER={DB_CONFIG['server']};"
            f"DATABASE={target_db};"
            f"Trusted_Connection=yes;TrustServerCertificate=yes;")

# --- 4. HEALTH CHECK ---
try:
    # Menggunakan 'db' untuk memanggil database utama di DB_CONFIG
    with pyodbc.connect(get_conn_str(db='db'), timeout=5) as conn:
        print(f"Connection to {DB_CONFIG['database']} successful.")
        print(f"Target Staging: {STAGING_TABLE}")
except Exception as e:
    print(f"Connection failed: {str(e)[:100]}")

Connection to OSHE_Analytics successful.
Target Staging: staging.accidents_raw


In [ ]:
print("Step 2: Syncing Data from MSHA Official Source")

MSHA_URL = "https://arlweb.msha.gov/OpenGovernmentData/DataSets/Accidents.zip"
ZIP_PATH = RAW_DIR / 'accidents_raw.zip'

try:
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(MSHA_URL, headers=headers, timeout=60)
    
    if response.status_code == 200:
        # Save ZIP binary
        with open(ZIP_PATH, 'wb') as f:
            f.write(response.content)
        
        # Extract content
        with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
            zip_ref.extractall(RAW_DIR)
            extracted_files = zip_ref.namelist()
        
        print(f"Extraction Complete. Files: {', '.join(extracted_files)}")
        print(f"Payload Size: {len(response.content) / (1024**2):.2f} MB")
    else:
        print(f"Source unavailable. HTTP Status: {response.status_code}")

except Exception as e:
    print(f"Critical error during acquisition: {e}")

In [8]:
print("Step 3: Data Ingestion & Schema Validation")

# Locate the extracted TXT file
target_files = list(RAW_DIR.glob('Accidents.[tT][xX][tT]'))

if not target_files:
    raise FileNotFoundError("Target file 'Accidents.txt' not found in raw directory.")

data_source = target_files[0]
print(f"Processing source: {data_source.name} ({data_source.stat().st_size / (1024**2):.2f} MB)")

try:
    # MSHA standard uses pipe delimiter
    df = pd.read_csv(data_source, sep='|', encoding='latin-1', low_memory=False, on_bad_lines='skip')
    
    print("Ingestion Success.")
    print(f"Dimensions: {df.shape[0]:,} rows | {df.shape[1]} features")
    
except Exception as e:
    print(f"Ingestion failed: {e}")

Step 3: Data Ingestion & Schema Validation
Processing source: Accidents.txt (214.66 MB)
Ingestion Success.
Dimensions: 270,921 rows | 57 features


In [20]:
print("Step 4 : Data Profiling and Staging Process")

# 1. Pipeline pembacaan data (Re-using logic from Step 3)
try:
    # Mengambil file TXT pertama yang ditemukan
    target_path = next(RAW_DIR.glob('Accidents.[tT][xX][tT]'))
    
    df = pd.read_csv(target_path, sep='|', encoding='latin-1', low_memory=False)
    
    # 2. Dataset Profiling
    print("-" * 30)
    print(f"Dataset Profile for: {target_path.name}")
    print(f"Total Records  : {len(df):,}")
    print(f"Total Features : {len(df.columns)}")
    print(f"Memory Usage   : {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print("-" * 30)

    # 3. Analisis Kualitas Data (Missing Values)
    null_summary = df.isnull().sum()
    top_missing = null_summary[null_summary > 0].sort_values(ascending=False).head(10)
    
    if not top_missing.empty:
        print("\nTop Missing Values:")
        for col, count in top_missing.items():
            print(f" - {col:<20}: {count:,} ({ (count/len(df))*100:.1f}%)")

    # 4. Save to Staging Area
    STAGING_FILE = STAGING_DIR / 'accidents_staging.csv'
    df.to_csv(STAGING_FILE, index=False)
    
    # 5. Export Data Dictionary Singkat
    DICT_FILE = STAGING_DIR / 'data_info.txt'
    with open(DICT_FILE, 'w') as f:
        f.write(f"Extraction Timestamp: {datetime.now()}\n")
        f.write(f"Total Rows: {len(df)}\n\n")
        f.write("Columns List:\n" + "\n".join([f"- {c}" for c in df.columns]))

    print(f"Process Completed. Staging saved to: {STAGING_FILE}")

except StopIteration:
    print("Error: Accidents.txt tidak ditemukan di folder raw.")
except Exception as e:
    print(f"Error: {e}")

Step 4 : Data Profiling and Staging Process
------------------------------
Dataset Profile for: Accidents.txt
Total Records  : 270,921
Total Features : 57
Memory Usage   : 679.13 MB
------------------------------

Top Missing Values:
 - CONTRACTOR_ID       : 242,826 (89.6%)
 - INVEST_BEGIN_DT     : 216,502 (79.9%)
 - EQUIP_MODEL_NO      : 163,123 (60.2%)
 - CLOSED_DOC_NO       : 150,913 (55.7%)
 - SCHEDULE_CHARGE     : 65,021 (24.0%)
 - DAYS_RESTRICT       : 61,957 (22.9%)
 - DAYS_LOST           : 46,418 (17.1%)
 - TOT_EXPER           : 44,004 (16.2%)
 - RETURN_TO_WORK_DT   : 42,791 (15.8%)
 - MINE_EXPER          : 41,117 (15.2%)
Process Completed. Staging saved to: e:\MyProject-GitHub\OSHE-Interview-Project\data\staging\accidents_staging.csv


In [21]:
print("Step 5: Database Profiling & Integrity Audit")

try:
    # 1. Data Retrieval - Menggunakan 'with' agar koneksi otomatis tertutup
    conn_str = get_conn_str(db='OSHE_Analytics') 
    with pyodbc.connect(conn_str) as conn:
        query = f"SELECT * FROM {STAGING_TABLE}"
        df = pd.read_sql(query, conn)

    print(f"Data Ingested: {len(df):,} records from {STAGING_TABLE}")

    # 2. Schema Audit
    # Menampilkan ringkasan tipe data yang masuk ke DataFrame
    print("\n[Schema Audit]")
    print(df.dtypes.value_counts().to_string())

    # 3. Data Integrity Check
    # Mengecek apakah ada baris yang sama persis (duplikat)
    duplicates = df.duplicated().sum()
    print(f"\n[Integrity Check]")
    print(f"  - Duplicate Records: {duplicates:,}")
    print(f"  - Clean Records: {len(df) - duplicates:,}")

    # 4. Statistical Summary (Numeric Only)
    # Mengambil kolom angka dan menampilkan statistik deskriptif dasar
    numeric_cols = df.select_dtypes(include=[np.number])
    if not numeric_cols.empty:
        print("\n[Metrics Summary - Top 10 Features]")
        print(numeric_cols.describe().T[['mean', 'min', 'max']].head(10))

    # 5. Export Snapshot for Validation
    # Menyimpan 100 baris pertama untuk keperluan inspeksi manual/Power BI preview
    SAMPLE_PATH = STAGING_DIR / 'data_sample.csv'
    df.head(100).to_csv(SAMPLE_PATH, index=False)
    print(f"Validation snapshot saved to: {SAMPLE_PATH}")

except Exception as e:
    print(f"Profiling Error: {e}")

Step 5: Database Profiling & Integrity Audit
Data Ingested: 270,921 records from staging.accidents_raw

[Schema Audit]
object            57
int64              1
datetime64[ns]     1

[Integrity Check]
  - Duplicate Records: 0
  - Clean Records: 270,921

[Metrics Summary - Top 10 Features]
            mean  min       max
STG_ID  135461.0  1.0  270921.0
Validation snapshot saved to: e:\MyProject-GitHub\OSHE-Interview-Project\data\staging\data_sample.csv


In [22]:
print("Cell 6: Defining OSHE Analytical Scope & Cleaning Strategy")

# Fokus pada fitur yang memiliki dampak langsung pada keselamatan (Safety Impact)
FEATURE_GROUPS = {
    'Core': ['DOCUMENT_NO', 'MINE_ID', 'OPERATOR_ID'],
    'Risk_Context': ['ACCIDENT_DT', 'CAL_YR', 'SUBUNIT', 'CLASSIFICATION', 'ACCIDENT_TYPE'],
    'Severity_Metrics': ['DEGREE_INJURY_CD', 'DAYS_LOST', 'SCHEDULE_CHARGE', 'NO_INJURIES'],
    'Human_Factors': ['OCCUPATION', 'ACTIVITY', 'TOT_EXPER', 'MINE_EXPER', 'JOB_EXPER']
}

# Mapping kategori untuk konversi tipe data otomatis
CLEANING_CONFIG = {
    'to_numeric': ['CAL_YR', 'CAL_QTR', 'NO_INJURIES', 'TOT_EXPER', 'DAYS_LOST', 'SCHEDULE_CHARGE'],
    'to_string': ['DEGREE_INJURY_CD', 'CLASSIFICATION_CD', 'ACCIDENT_TYPE_CD', 'MINE_ID'],
    'missing_threshold': 0.70  # Buang kolom jika data kosong > 70%
}

all_important_cols = [col for sublist in FEATURE_GROUPS.values() for col in sublist]
print(f"Strategy Initialized. Monitoring {len(all_important_cols)} critical features.")

Cell 6: Defining OSHE Analytical Scope & Cleaning Strategy
Strategy Initialized. Monitoring 17 critical features.


In [23]:
print("Cell 7: Executing Data Cleaning Pipeline")

df_clean = df.copy()

# 1. Dimensionality Reduction (Drop High Missing)
null_pct = df_clean.isnull().mean()
cols_to_drop = [c for c in df_clean.columns if null_pct[c] > CLEANING_CONFIG['missing_threshold'] 
                and c not in all_important_cols]
df_clean.drop(columns=cols_to_drop, inplace=True)

# 2. Type Casting
for col in CLEANING_CONFIG['to_numeric']:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').fillna(0)

# 3. Text Standardization
# Membersihkan spasi dan mengubah 'nan' atau '?' menjadi 'Unknown' secara massal
obj_cols = df_clean.select_dtypes(include=['object']).columns
df_clean[obj_cols] = df_clean[obj_cols].astype(str).apply(lambda x: x.str.strip())
df_clean.replace(['nan', '?', 'None', 'No Value Found'], 'Unknown', inplace=True)

# 4. Feature Engineering (New Insights)
if 'ACCIDENT_DT' in df_clean.columns:
    dt_series = pd.to_datetime(df_clean['ACCIDENT_DT'], errors='coerce')
    df_clean['MONTH'] = dt_series.dt.month
    df_clean['DAY_OF_WEEK'] = dt_series.dt.day_name()

# Menghitung SEVERITY_SCORE sebagai kombinasi hari hilang dan denda jadwal
df_clean['SEVERITY_SCORE'] = df_clean['DAYS_LOST'] + df_clean['SCHEDULE_CHARGE']
df_clean['IS_FATAL'] = (df_clean['DEGREE_INJURY_CD'] == '01').astype(int)

print(f"Cleaning Complete. Remaining Nulls: {df_clean.isnull().sum().sum()}")

Cell 7: Executing Data Cleaning Pipeline
Cleaning Complete. Remaining Nulls: 0


In [24]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

print("Cell 8: Preparing Features for Machine Learning (Clustering)")

# 1. Feature Selection
# Pilih fitur numerik dan kategorik yang paling berpengaruh pada pola kecelakaan
CLUSTER_FEATURES = [
    'CAL_YR', 'MONTH', 'SEVERITY_SCORE', 'IS_FATAL', 
    'NO_INJURIES', 'TOT_EXPER', 'MINE_EXPER', 'JOB_EXPER'
]

df_ml = df_clean[CLUSTER_FEATURES].copy()

# 2. Categorical Encoding
# Mengubah teks menjadi angka karena algoritma clustering hanya menerima angka
le = LabelEncoder()
for col in df_ml.select_dtypes(include=['object']).columns:
    df_ml[col] = le.fit_transform(df_ml[col])

# 3. Feature Scaling (Penting untuk K-Means)
# Menyamakan skala data (misal: Tahun 2024 vs Jumlah Cedera 1) agar tidak jomplang
scaler = StandardScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df_ml),
    columns=df_ml.columns
)

print(f"Features Ready: {df_scaled.shape[1]} variables scaled.")
print(df_scaled.head(3))

Cell 8: Preparing Features for Machine Learning (Clustering)
Features Ready: 8 variables scaled.
     CAL_YR     MONTH  SEVERITY_SCORE  IS_FATAL  NO_INJURIES  TOT_EXPER  \
0  0.467442  0.481290       -0.139006 -0.066533     0.154529  -0.760775   
1  2.002024 -0.714448       -0.139006 -0.066533     0.154529   1.417391   
2 -0.648618  1.378094       -0.139006 -0.066533     0.154529   2.395987   

   MINE_EXPER  JOB_EXPER  
0   -0.990713  -1.039982  
1    0.906909   1.173644  
2    0.645008   0.648729  


In [26]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

print("Cell 9: Execute Optimized Clustering Pipeline")

# 1. Hyperparameter Tuning (Elbow & Silhouette Method) via Sampling
# Menggunakan 20k sample agar proses pencarian 'k' cepat namun tetap representatif
df_sample = df_scaled.sample(n=min(20000, len(df_scaled)), random_state=42)
k_results = []

for k in range(3, 7):
    model = KMeans(n_clusters=k, n_init=5, max_iter=100, random_state=42)
    labels = model.fit_predict(df_sample)
    sil = silhouette_score(df_sample, labels, sample_size=5000)
    k_results.append((k, model.inertia_, sil))
    print(f" - Testing k={k}: Silhouette Score = {sil:.3f}")

# 2. Select Best K and Fit to Full Dataset
best_k = max(k_results, key=lambda x: x[2])[0]
print(f"\n✓ Optimal Cluster Selected: {best_k}")

# Fitting model final ke seluruh data (270k+ baris)
final_kmeans = KMeans(n_clusters=best_k, n_init=10, random_state=42)
df_clean['CLUSTER'] = final_kmeans.fit_predict(df_scaled)

# 3. Quick Distribution Check
print("\n[Cluster Distribution]")
print(df_clean['CLUSTER'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%')

# 4. Mapping Cluster ke Label Bisnis yang Informatif
cluster_map = {
    0: 'Moderate Severity',
    1: 'Low Risk - Experienced',
    2: 'High Risk - Severe Injuries',
    3: 'Low Risk - New Workers',
    4: 'CRITICAL - Fatal Risk'
}

# Membuat kolom label dan mengisi nilai yang kosong (jika ada) dengan 'Unclassified'
df_clean['CLUSTER_LABEL'] = df_clean['CLUSTER'].map(cluster_map).fillna('Unclassified')

# Menentukan RISK_LEVEL secara otomatis
def assign_risk(cluster):
    if cluster == 4: return 'Critical'
    if cluster in [0, 2]: return 'High'
    return 'Low'

df_clean['RISK_LEVEL'] = df_clean['CLUSTER'].apply(assign_risk)

# FINAL CHECK: Pastikan tidak ada kolom penting yang masih Null
df_final = df_clean.fillna('Unknown')

print(f"✅ Blank Labels Fixed. Unique Labels: {df_final['CLUSTER_LABEL'].unique()}")

Cell 9: Execute Optimized Clustering Pipeline
 - Testing k=3: Silhouette Score = 0.258
 - Testing k=4: Silhouette Score = 0.266
 - Testing k=5: Silhouette Score = 0.267
 - Testing k=6: Silhouette Score = 0.239

✓ Optimal Cluster Selected: 5

[Cluster Distribution]
CLUSTER
3    47.09%
0    28.31%
1    23.74%
4     0.44%
2     0.41%
Name: proportion, dtype: object
✅ Blank Labels Fixed. Unique Labels: ['Low Risk - New Workers' 'Low Risk - Experienced' 'Moderate Severity'
 'High Risk - Severe Injuries' 'CRITICAL - Fatal Risk']


In [27]:
print("Cell 10: Loading Clustered Insights to Production Table")

# 1. Feature Selection for Production
# Memilih kolom asli ditambah label cluster untuk kebutuhan Dashboard Power BI
export_cols = all_important_cols + ['YEAR', 'MONTH', 'DAY_OF_WEEK', 'SEVERITY_SCORE', 'IS_FATAL', 'CLUSTER']
df_final = df_clean[[c for c in export_cols if c in df_clean.columns]].copy()

# 2. SQL Production Schema Handling
try:
    with pyodbc.connect(get_conn_str()) as conn:
        cursor = conn.cursor()
        
        # Reset table jika sudah ada (Full Refresh)
        cursor.execute(f"IF OBJECT_ID('{PRODUCTION_TABLE}') IS NOT NULL DROP TABLE {PRODUCTION_TABLE}")
        
        # Logic sederhana pembuatan tabel: Semua string jadi NVARCHAR, angka jadi FLOAT/INT
        cols_sql = []
        for col, dtype in df_final.dtypes.items():
            sql_type = "INT" if "int" in str(dtype) else "FLOAT" if "float" in str(dtype) else "NVARCHAR(MAX)"
            cols_sql.append(f"[{col}] {sql_type}")
            
        cursor.execute(f"CREATE TABLE {PRODUCTION_TABLE} ({', '.join(cols_sql)})")
        conn.commit()
        
        # 3. Batch Loading (Optimized for 270k records)
        print(f"Inserting {len(df_final):,} rows...")
        placeholders = ", ".join(["?"] * len(df_final.columns))
        sql_insert = f"INSERT INTO {PRODUCTION_TABLE} VALUES ({placeholders})"
        
        # Menggunakan list of tuples untuk kecepatan insert (executemany)
        data_tuples = [tuple(x) for x in df_final.values]
        cursor.executemany(sql_insert, data_tuples)
        conn.commit()

    print(f"Production Table Created: {PRODUCTION_TABLE}")

except Exception as e:
    print(f"Failed to load production: {e}")

Cell 10: Loading Clustered Insights to Production Table
Inserting 270,921 rows...
Production Table Created: dbo.Accidents_Clustered_Final


In [28]:
print("Cell 11: Analysis of Cluster Profiles")

# 1. Aggregate Statistics per Cluster
cluster_stats = df_clean.groupby('CLUSTER').agg({
    'IS_FATAL': 'mean',
    'SEVERITY_SCORE': 'mean',
    'TOT_EXPER': 'mean',
    'DOCUMENT_NO': 'count'
}).rename(columns={'DOCUMENT_NO': 'Record_Count'})

# 2. Automated Labeling Logic (Professional Interpretation)
def label_cluster(row):
    if row['IS_FATAL'] > 0.02: return "High Severity / Fatal Risk"
    if row['TOT_EXPER'] < 5:  return "High Risk / New Workers"
    if row['SEVERITY_SCORE'] > 50: return "Intermediate / Severe Injuries"
    return "Standard Risk / Experienced"

cluster_stats['Profile_Name'] = cluster_stats.apply(label_cluster, axis=1)

# 3. Presentation Output
print("-" * 80)
print(f"{'ID':<4} {'Profile Name':<35} {'Size':<10} {'Fatal%':<8} {'Avg_Exp':<8}")
print("-" * 80)

for idx, row in cluster_stats.iterrows():
    print(f"{idx:<4} {row['Profile_Name']:<35} {int(row['Record_Count']):<10,} "
          f"{row['IS_FATAL']*100:<8.2f}% {row['TOT_EXPER']:<8.1f}y")

print("-" * 80)
print(f"Overall Clustering Quality: { 'ACCEPTABLE' if best_k > 2 else 'NEEDS TUNING' }")

Cell 11: Analysis of Cluster Profiles
--------------------------------------------------------------------------------
ID   Profile Name                        Size       Fatal%   Avg_Exp 
--------------------------------------------------------------------------------
0    High Risk / New Workers             76,700     0.00    % 3.4     y
1    Standard Risk / Experienced         64,319     0.00    % 25.6    y
2    Intermediate / Severe Injuries      1,124      0.00    % 10.9    y
3    High Risk / New Workers             127,584    0.00    % 4.4     y
4    High Severity / Fatal Risk          1,194      100.00  % 13.1    y
--------------------------------------------------------------------------------
Overall Clustering Quality: ACCEPTABLE


In [29]:
import sqlalchemy as sa
import urllib

print("Step 12: Final Business Labeling & SQL Production Export")

# 1. Interpretasi Bisnis untuk Cluster
# Memberikan narasi pada hasil clustering agar informatif di Power BI
risk_mapping = {
    0: "Moderate Severity",
    1: "Low Risk - Operational", 
    2: "CRITICAL - Fatal Risk",
    3: "Low Risk - Maintenance"
}

# Mapping label dan level risiko
df_clean['CLUSTER_LABEL'] = df_clean['CLUSTER'].map(risk_mapping)
df_clean['RISK_LEVEL'] = df_clean['CLUSTER'].apply(lambda x: 'High' if x == 2 else 'Medium' if x == 0 else 'Low')

# 2. Seleksi Kolom Produksi
# Hanya membawa kolom yang relevan untuk visualisasi dashboard
final_features = [
    'MINE_ID', 'DOCUMENT_NO', 'ACCIDENT_DT', 'YEAR', 'MONTH', 'DAY_OF_WEEK', 'SUBUNIT', 'CLASSIFICATION', 'ACCIDENT_TYPE', 'DEGREE_INJURY', 'SEVERITY_SCORE', 'IS_FATAL', 'TOT_EXPER', 'CLUSTER_LABEL', 'RISK_LEVEL', 'FIPS_STATE_CD'
]
df_final = df_clean[[c for c in final_features if c in df_clean.columns]].copy()

# 3. High-Performance SQL Export
try:
    # PERBAIKAN: Menggunakan get_conn_str(db='db') yang sudah didefinisikan di Cell 1
    # fast_executemany=True sangat penting untuk kecepatan insert data besar
    conn_params = urllib.parse.quote_plus(get_conn_str(db='db')) 
    engine = sa.create_engine(f"mssql+pyodbc:///?odbc_connect={conn_params}", fast_executemany=True)
    
    print(f"Transferring {len(df_final):,} records to SQL Server...")
    
    # Export menggunakan SQLAlchemy (Jauh lebih cepat dari pyodbc executemany biasa)
    df_final.to_sql(
        name='Accidents_Clustered_Final', 
        con=engine, 
        if_exists='replace', 
        index=False,
        chunksize=25000 
    )
    
    print(f"SUCCESS: Table 'Accidents_Clustered_Final' is ready in {DB_CONFIG['database']}.")

except Exception as e:
    print(f"SQL Export Failed: {e}")

Step 12: Final Business Labeling & SQL Production Export
Transferring 270,921 records to SQL Server...
SUCCESS: Table 'Accidents_Clustered_Final' is ready in OSHE_Analytics.
